<a href="https://colab.research.google.com/github/laribar/bitcoinprediction/blob/main/Bot_Funcional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=e66311d4a75ecd1bb1972cabe48b075052b776d229a801a5ef24b495bac9cad4
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta


In [32]:
import yfinance as yf
import numpy as np
import pandas as pd
import ta
import requests
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 1. Configurações
ASSETS = ["BTC-USD", "ETH-USD", "BNB-USD", "SOL-USD", "XRP-USD"]
TIMEFRAMES = [
    {"interval": "15m", "period": "30d", "atr": 0.02},
    {"interval": "1h", "period": "90d", "atr": 0.03},
    {"interval": "1d", "period": "1000d", "atr": 0.05}
]
TELEGRAM_TOKEN = "8044593190:AAFtUWYHd3uqd-AtQi3uqg42F9G6uV95v8k"
TELEGRAM_CHAT_ID = "-4744645054"

# 2. Obtenção de dados

def get_stock_data(asset, interval="15m", period="700d"):
    data = yf.download(asset, period=period, interval=interval, progress=False, auto_adjust=False)
    if isinstance(data.columns, pd.MultiIndex):
        data.columns = data.columns.get_level_values(0)
    data.columns = [col.split()[-1] if " " in col else col for col in data.columns]
    # Remove colunas duplicadas ou com sufixos como 'Adj Close'
    data = data.loc[:, ~data.columns.duplicated()]
    col_map = {}
    for col in data.columns:
        for std_col in ["Open", "High", "Low", "Close", "Adj Close", "Volume"]:
            if std_col.lower() in col.lower():
                col_map[col] = std_col
    data = data.rename(columns=col_map)
    # Garante que apenas as colunas necessárias estejam presentes
    data = data[["Open", "High", "Low", "Close", "Volume"]]
    required = ["Open", "High", "Low", "Close", "Volume"]
    if not all(col in data.columns for col in required):
        raise ValueError(f"⚠️ Dados de {asset} não possuem todas as colunas necessárias.")
    return data

# 3. Indicadores

def calculate_indicators(data):
    data = data.copy()
    data.reset_index(drop=True, inplace=True)

    for col in ["Open", "High", "Low", "Close", "Volume"]:
        if isinstance(data[col], pd.DataFrame):
            data[col] = data[col].iloc[:, 0]
        elif isinstance(data[col].iloc[0], (list, np.ndarray)):
            data[col] = data[col].apply(lambda x: x[0] if isinstance(x, (list, np.ndarray)) else x)
        data[col] = data[col].astype(float)

    # Indicadores
    data["RSI"] = ta.momentum.RSIIndicator(close=data["Close"], window=14).rsi()
    data["SMA_50"] = ta.trend.SMAIndicator(close=data["Close"], window=50).sma_indicator()
    data["SMA_200"] = ta.trend.SMAIndicator(close=data["Close"], window=200).sma_indicator()

    macd = ta.trend.MACD(close=data["Close"])
    data["MACD"] = macd.macd()
    data["MACD_Signal"] = macd.macd_signal()

    bb = ta.volatility.BollingerBands(close=data["Close"], window=20)
    data["Bollinger_Upper"] = bb.bollinger_hband()
    data["Bollinger_Lower"] = bb.bollinger_lband()

    adx = ta.trend.ADXIndicator(high=data["High"], low=data["Low"], close=data["Close"], window=14)
    data["ADX"] = adx.adx()

    stoch = ta.momentum.StochasticOscillator(high=data["High"], low=data["Low"], close=data["Close"], window=14)
    data["Stoch_K"] = stoch.stoch()
    data["Stoch_D"] = stoch.stoch_signal()

    data["TP"] = (data["High"] + data["Low"] + data["Close"]) / 3
    data["VWAP"] = (data["TP"] * data["Volume"]).cumsum() / (data["Volume"].replace(0, np.nan).cumsum())
    data.drop("TP", axis=1, inplace=True)

    data["Doji"] = ((abs(data["Close"] - data["Open"]) / (data["High"] - data["Low"] + 1e-9)) < 0.1).astype(int)
    data["Engulfing"] = ((data["Open"].shift(1) > data["Close"].shift(1)) & (data["Open"] < data["Close"]) &
                          (data["Close"] > data["Open"].shift(1)) & (data["Open"] < data["Close"].shift(1))).astype(int)
    data["Hammer"] = (((data["High"] - data["Low"]) > 3 * abs(data["Open"] - data["Close"])) &
                       ((data["Close"] - data["Low"]) / (data["High"] - data["Low"] + 1e-9) > 0.6) &
                       ((data["Open"] - data["Low"]) / (data["High"] - data["Low"] + 1e-9) > 0.6)).astype(int)

    data.dropna(inplace=True)
    return data


# 4. Modelo

def train_ml_model(data, verbose=False):
    if len(data) < 100:
        return None

    df = data.copy()
    df["Signal"] = np.select([
        (df["RSI"] < 40) & (df["SMA_50"] > df["SMA_200"]),
        (df["RSI"] > 60) & (df["SMA_50"] < df["SMA_200"])
    ], [1, -1], default=0)

    features = get_feature_columns()
    X = df[features]
    y = df["Signal"]

    signal_count = (y == 1).sum()
    if verbose:
        print(f"📉 Sinais de COMPRA encontrados: {signal_count} em {len(y)} candles")

    if len(np.unique(y)) < 2:
        return None

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=False)
    if len(np.unique(y_train)) < 2:
        return None

    model = RandomForestClassifier(n_estimators=200, max_depth=8, class_weight='balanced', random_state=42)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_val)
    report = classification_report(y_val, y_pred, output_dict=True, zero_division=0)

    model.validation_score = {
        "accuracy": report.get("accuracy"),
        "precision": report.get("1", {}).get("precision"),
        "recall": report.get("1", {}).get("recall"),
        "f1": report.get("1", {}).get("f1-score")
    }

    return model

# 5. Utilitários

def get_feature_columns():
    return [
        "RSI", "MACD", "MACD_Signal", "SMA_50", "SMA_200", "Bollinger_Upper",
        "Bollinger_Lower", "ADX", "Stoch_K", "Stoch_D", "VWAP",
        "Doji", "Engulfing", "Hammer"
    ]

def generate_explanation(row, prediction):
    explanation = []
    if prediction == 1:
        explanation.append("🟢 O modelo prevê uma tendência de ALTA.")
    elif prediction == -1:
        explanation.append("🔴 O modelo prevê uma tendência de BAIXA.")
    else:
        explanation.append("⚪ Sinal neutro.")

    if row["RSI"] < 30:
        explanation.append("🔽 RSI abaixo de 30 indica sobrevenda.")
    elif row["RSI"] > 70:
        explanation.append("🔼 RSI acima de 70 indica sobrecompra.")

    if row["SMA_50"] > row["SMA_200"]:
        explanation.append("📈 SMA 50 acima da 200, tendência de alta.")
    else:
        explanation.append("📉 SMA 50 abaixo da 200, tendência de baixa.")

    if row["MACD"] > row["MACD_Signal"]:
        explanation.append("💹 MACD cruzando para cima, possível reversão positiva.")
    else:
        explanation.append("🔻 MACD abaixo da linha de sinal.")

    if row["Doji"] == 1:
        explanation.append("⚠️ Padrão de candle Doji detectado (possível reversão).")

    if row["Engulfing"] == 1:
        explanation.append("📊 Padrão de engolfo detectado (sinal forte de reversão).")

    return "\n".join(explanation)

def calculate_targets(current_price, direction, atr=0.02):
    if direction == 1:
        return {
            "TP1": round(current_price * (1 + atr), 2),
            "TP2": round(current_price * (1 + atr * 2), 2),
            "SL": round(current_price * (1 - atr), 2)
        }
    elif direction == -1:
        return {
            "TP1": round(current_price * (1 - atr), 2),
            "TP2": round(current_price * (1 - atr * 2), 2),
            "SL": round(current_price * (1 + atr), 2)
        }
    else:
        return {"TP1": None, "TP2": None, "SL": None}

def send_telegram_message(message):
    url = f"https://api.telegram.org/bot{TELEGRAM_TOKEN}/sendMessage"
    payload = {"chat_id": TELEGRAM_CHAT_ID, "text": message, "parse_mode": "HTML"}
    requests.post(url, json=payload)

# 6. Execução

def run_analysis():
    for asset in ASSETS:
        print(f"\n📊 Analisando {asset}...")
        models = {}
        data = {}

        try:
            for tf in TIMEFRAMES:
                interval = tf['interval']
                period = tf['period']
                data[interval] = calculate_indicators(get_stock_data(asset, interval, period))
                models[interval] = train_ml_model(data[interval], verbose=True)
        except Exception as e:
            print(f"❌ Erro ao processar {asset}: {e}")
            continue

        if all(model is None for model in models.values()):
            print(f"⚠️ Nenhum modelo foi treinado para {asset}.")
            continue

        current_price = data["15m"]["Close"].iloc[-1]
        message = f"🔍 {asset}\n💰 Preço atual: ${current_price:.2f}\n"

        for tf in TIMEFRAMES:
            interval = tf['interval']
            model = models.get(interval)

            if model is None:
                message += f"\n⏱️ {interval}: Dados insuficientes para este timeframe.\n"
                continue

            latest_data = data[interval].iloc[-1]
            features = get_feature_columns()
            input_data = pd.DataFrame([latest_data[features]])
            prediction = model.predict(input_data)[0]
            proba = model.predict_proba(input_data)[0][np.where(model.classes_ == prediction)[0][0]]

            def safe_format(val): return f"{val:.2f}" if val is not None else "N/A"
            score = model.validation_score
            score_text = f"📈 Accuracy: {safe_format(score['accuracy'])} | Precision: {safe_format(score['precision'])} | Recall: {safe_format(score['recall'])}"

            targets = calculate_targets(current_price, prediction, tf['atr'])
            explanation = generate_explanation(latest_data, prediction)

            time_label = {"15m": "15 minutos", "1h": "1 hora", "1d": "Diário"}[interval]

            message += f"""
⏱️ {time_label}:
{'🟢 COMPRA' if prediction == 1 else ('🔴 VENDA' if prediction == -1 else '⚪ NEUTRO')} com {proba*100:.1f}% de confiança
🎯 TP1: ${targets['TP1']} | TP2: ${targets['TP2']} | SL: ${targets['SL']}
📌 Previsão: ${current_price * (1 + (tf['atr'] if prediction == 1 else (-tf['atr'] if prediction == -1 else 0))):.2f}
{score_text}
{explanation}
"""

        send_telegram_message(message)
        print(message)

if __name__ == "__main__":
    input("\n🚀 Pressione Enter para iniciar a análise...\n")
    run_analysis()



🚀 Pressione Enter para iniciar a análise...


📊 Analisando BTC-USD...
📉 Sinais de COMPRA encontrados: 191 em 2291 candles
📉 Sinais de COMPRA encontrados: 110 em 929 candles
📉 Sinais de COMPRA encontrados: 80 em 801 candles
🔍 BTC-USD
💰 Preço atual: $87434.34

⏱️ 15 minutos:
⚪ NEUTRO com 80.7% de confiança
🎯 TP1: $None | TP2: $None | SL: $None
📌 Previsão: $87434.34
📈 Accuracy: 0.86 | Precision: 0.56 | Recall: 0.39
⚪ Sinal neutro.
📈 SMA 50 acima da 200, tendência de alta.
💹 MACD cruzando para cima, possível reversão positiva.

⏱️ 1 hora:
⚪ NEUTRO com 96.0% de confiança
🎯 TP1: $None | TP2: $None | SL: $None
📌 Previsão: $87434.34
📈 Accuracy: 0.89 | Precision: 0.00 | Recall: 0.00
⚪ Sinal neutro.
📈 SMA 50 acima da 200, tendência de alta.
🔻 MACD abaixo da linha de sinal.

⏱️ Diário:
⚪ NEUTRO com 90.0% de confiança
🎯 TP1: $None | TP2: $None | SL: $None
📌 Previsão: $87434.34
📈 Accuracy: 0.89 | Precision: 0.00 | Recall: 0.00
⚪ Sinal neutro.
📈 SMA 50 acima da 200, tendência de alta.
💹 MACD cruzan